In [ ]:
from d123.dataset.scene.scene_builder import ArrowSceneBuilder
from d123.dataset.scene.scene_filter import SceneFilter

from d123.common.multithreading.worker_sequential import Sequential

# from d123.common.multithreading.worker_ray import RayDistributed

In [ ]:
import os, psutil


def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Memory usage: {memory_info.rss / 1024 ** 2:.2f} MB")

split = "nuplan_mini_val"
# split = "carla"

# log_names = ["2021.06.07.12.54.00_veh-35_01843_02314"]
scene_tokens = None
# scene_tokens = ["2283aea39bc1505e"]
log_names = None

scene_filter = SceneFilter(
    split_names=[split],
    log_names=log_names,
    scene_tokens=scene_tokens,
    duration_s=15.1,
    history_s=1.0,
)
scene_builder = ArrowSceneBuilder("/home/daniel/d123_workspace/data")
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print("num scenes found", len(scenes))

In [ ]:
from typing import Tuple
d123.datatypes.detections.detection import BoxDetection
d123.datatypes.detections.detection_types import DYNAMIC_DETECTION_TYPES, STATIC_DETECTION_TYPES
from d123.geometry import StateSE2
from d123.geometry.transform.tranform_2d import translate_along_yaw
from d123.geometry.vector import Vector2D
from d123.common.visualization.matplotlib.observation import (
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_ego_vehicle_to_ax,
    add_traffic_lights_to_ax,
)
from d123.dataset.scene.abstract_scene import AbstractScene

import matplotlib.pyplot as plt


def _plot_scene_on_ax(ax: plt.Axes, scene: AbstractScene, iteration: int = 0, radius: float = 80) -> plt.Axes:

    ego_vehicle_state = scene.get_ego_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)
    traffic_light_detections = scene.get_traffic_light_detections_at_iteration(iteration)
    map_api = scene.get_map_api()

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d
    add_default_map_on_ax(ax, map_api, point_2d, radius=radius, route_lane_group_ids=None)
    # add_traffic_lights_to_ax(ax, traffic_light_detections, map_api)

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    ax.set_xlim(point_2d.x - radius, point_2d.x + radius)
    ax.set_ylim(point_2d.y - radius, point_2d.y + radius)

    ax.set_aspect("equal", adjustable="box")
    return ax


iteration = 20
scene = scenes[20]

fig, ax = plt.subplots(figsize=(10, 10))
_plot_scene_on_ax(ax, scene, iteration, 80)

box_detections = scene.get_box_detections_at_iteration(iteration)


def _get_dxy(box_detection: BoxDetection) -> Tuple[float, float]:
    """Get the change in x and y coordinates from the box detection."""
    center = box_detection.center.state_se2
    # endpoint_x = translate_along_yaw(center, Vector2D(abs(box_detection.velocity.x), 0.0))
    # endpoint_y = translate_along_yaw(center, Vector2D(0.0, abs(box_detection.velocity.y)))
    # print(box_detection.velocity.x, box_detection.velocity.y)

    endpoint_x = StateSE2(center.x + box_detection.velocity.x, center.y, center.yaw)
    endpoint_y = StateSE2(center.x, center.y + box_detection.velocity.y, center.yaw)

    return endpoint_x, endpoint_y


for box_detection in box_detections:
    if box_detection.metadata.detection_type in STATIC_DETECTION_TYPES:
        continue
    endpoint_x, endpoint_y = _get_dxy(box_detection)
    ax.annotate(
        "",
        xytext=(box_detection.center.state_se2.point_2d.x, box_detection.center.state_se2.point_2d.y),
        xy=(endpoint_x.x, endpoint_x.y),
        arrowprops=dict(arrowstyle="->"),
    )
    ax.annotate(
        "",
        xytext=(box_detection.center.state_se2.point_2d.x, box_detection.center.state_se2.point_2d.y),
        xy=(endpoint_y.x, endpoint_y.y),
        arrowprops=dict(arrowstyle="->"),
    )